# 📊 ANOVA - Analysis of Variance
## Comparing Means Across Multiple Groups

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/The-Pattern-Hunter/interactive-ecology-biometry/blob/main/unit-4-biometry/notebooks/09_anova.ipynb)

---

> *"The analysis of variance is not a mathematical theorem, but rather a convenient method of arranging the arithmetic."* - R.A. Fisher

### 🎯 Learning Objectives

By the end of this notebook, you will:
1. Understand **when to use ANOVA** vs t-tests
2. Calculate and interpret **one-way ANOVA**
3. Partition **total variance** into components
4. Perform **post-hoc tests** for pairwise comparisons
5. Apply **two-way ANOVA** for factorial designs
6. Interpret **main effects** and **interactions**
7. Use **repeated measures ANOVA** for within-subjects designs
8. Check **ANOVA assumptions** with diagnostics
9. Apply ANOVA to **real ecological data**

In [ ]:
# Setup
!pip install numpy pandas plotly matplotlib scipy scikit-learn statsmodels -q

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import itertools
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

print("✅ Ready for ANOVA!")
print("📊 Let's compare means across groups!")

---

## 📚 Part 1: What is ANOVA?

### Definition:

**ANOVA (Analysis of Variance)**: Statistical test for comparing means of **3 or more groups** simultaneously.

### Why Not Multiple t-tests?

**Problem**: Type I error inflation

**Example**: Compare 4 groups
```
Pairwise comparisons needed:
A vs B
A vs C
A vs D
B vs C
B vs D
C vs D
─────────
Total: 6 tests!

If α = 0.05 for each test:
Probability of at least one false positive:
1 - (0.95)⁶ = 0.265 (26.5%!)
```

**Solution**: ANOVA tests all groups simultaneously
- Single test with α = 0.05
- Controls family-wise error rate

### When to Use ANOVA:

**✅ Use ANOVA when**:
- Comparing **3+ groups**
- **One continuous** dependent variable (Y)
- **One or more categorical** independent variables (factors)
- Want to test **overall difference** first

**❌ Use t-test when**:
- Comparing **only 2 groups**

### Types of ANOVA:

#### **1. One-Way ANOVA**
```
One factor with 3+ levels
Example: Compare plant growth across 4 fertilizer types
```

#### **2. Two-Way ANOVA**
```
Two factors
Example: Fertilizer (4 types) × Watering (2 levels)
Can test interactions!
```

#### **3. Repeated Measures ANOVA**
```
Same subjects measured multiple times
Example: Plant height measured weekly over 8 weeks
```

#### **4. ANCOVA** (Analysis of Covariance)
```
ANOVA + Regression
Control for continuous covariate
Example: Compare fertilizers, controlling for initial size
```

### The Core Logic:

**ANOVA partitions total variance**:

```
Total Variance = Between-Group Variance + Within-Group Variance
      (SST)     =        (SSB)           +       (SSW)
```

**If groups truly differ**:
- Between-group variance will be LARGE
- Within-group variance will be SMALL
- **F-ratio will be large**

**F-statistic**:
```
F = Between-group variance / Within-group variance
  = MSB / MSW
```

### Visual Representation:

```
GROUPS DIFFER (Large F):

Group A    Group B    Group C
  •••        •••        •••
  •••        •••        •••
  •••        •••        •••
───────────────────────────
Large between-group variance
Small within-group variance


GROUPS SIMILAR (Small F):

     All Groups
    •  •  •  •
  •  •  •  •  •
    •  •  •  •
───────────────────────────
Small between-group variance
Large within-group variance
```

---

## 📊 Part 2: One-Way ANOVA

### The Model:

```
Yᵢⱼ = μ + αᵢ + εᵢⱼ
```

Where:
- **Yᵢⱼ** = Observation j in group i
- **μ** = Grand mean (overall average)
- **αᵢ** = Effect of group i
- **εᵢⱼ** = Random error

### Hypotheses:

**Null Hypothesis (H₀)**:
```
All group means are equal
H₀: μ₁ = μ₂ = μ₃ = ... = μₖ
```

**Alternative Hypothesis (Hₐ)**:
```
At least one mean differs
Hₐ: Not all μᵢ are equal
```

### ANOVA Table:

| Source | SS | df | MS | F | p-value |
|--------|----|----|-------|---|--------|
| **Between Groups** | SSB | k-1 | MSB = SSB/(k-1) | F = MSB/MSW | p |
| **Within Groups** | SSW | N-k | MSW = SSW/(N-k) | | |
| **Total** | SST | N-1 | | | |

Where:
- **k** = Number of groups
- **N** = Total sample size
- **SS** = Sum of squares
- **df** = Degrees of freedom
- **MS** = Mean square (SS/df)

### Calculating Sums of Squares:

**Total Sum of Squares (SST)**:
```
SST = Σ(Yᵢⱼ - Ȳ)²
```
Total deviation from grand mean

**Between-Groups Sum of Squares (SSB)**:
```
SSB = Σnᵢ(Ȳᵢ - Ȳ)²
```
Group means' deviation from grand mean

**Within-Groups Sum of Squares (SSW)**:
```
SSW = Σ(Yᵢⱼ - Ȳᵢ)²
```
Individual values' deviation from group means

**Relationship**:
```
SST = SSB + SSW
```

### Effect Size:

**Eta-squared (η²)**:
```
η² = SSB / SST
```

**Interpretation**:
- η² = 0.01: Small effect
- η² = 0.06: Medium effect
- η² = 0.14: Large effect

**Meaning**: Proportion of total variance explained by group differences

In [ ]:
# One-way ANOVA example: Plant growth under different fertilizers
def oneway_anova_example(n_per_group=15, seed=42):
    """
    Compare plant height across 4 fertilizer treatments
    """
    np.random.seed(seed)
    
    # Generate data for 4 fertilizer types
    treatments = ['Control', 'Organic', 'NPK', 'Slow-release']
    means = [25, 30, 35, 32]  # True group means
    std = 4  # Common standard deviation
    
    data = []
    for treatment, mean in zip(treatments, means):
        heights = np.random.normal(mean, std, n_per_group)
        for height in heights:
            data.append({'Treatment': treatment, 'Height': height})
    
    df = pd.DataFrame(data)
    
    # Perform one-way ANOVA
    groups = [df[df['Treatment'] == t]['Height'].values for t in treatments]
    f_stat, p_value = stats.f_oneway(*groups)
    
    # Calculate effect size (eta-squared)
    # Calculate sums of squares manually
    grand_mean = df['Height'].mean()
    sst = np.sum((df['Height'] - grand_mean)**2)
    
    group_means = df.groupby('Treatment')['Height'].mean()
    ssb = sum([n_per_group * (mean - grand_mean)**2 for mean in group_means])
    
    ssw = sst - ssb
    eta_squared = ssb / sst
    
    # Create visualization
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            'Distribution by Treatment',
            'Mean ± SE'
        ),
        horizontal_spacing=0.15
    )
    
    # Box plots
    colors = ['lightblue', 'lightgreen', 'lightcoral', 'gold']
    for i, treatment in enumerate(treatments):
        subset = df[df['Treatment'] == treatment]['Height']
        fig.add_trace(
            go.Box(
                y=subset,
                name=treatment,
                marker_color=colors[i],
                boxmean='sd',
                showlegend=False
            ),
            row=1, col=1
        )
    
    # Bar plot with error bars
    group_stats = df.groupby('Treatment')['Height'].agg(['mean', 'sem'])
    
    fig.add_trace(
        go.Bar(
            x=treatments,
            y=group_stats['mean'],
            error_y=dict(
                type='data',
                array=group_stats['sem'],
                visible=True
            ),
            marker_color=colors,
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Add ANOVA results
    fig.add_annotation(
        text=f"<b>One-Way ANOVA Results:</b><br>" +
             f"F({len(treatments)-1}, {len(df)-len(treatments)}) = {f_stat:.2f}<br>" +
             f"p-value = {p_value:.4f}<br>" +
             f"η² = {eta_squared:.3f}<br><br>" +
             ("<b>Significant difference!</b>" if p_value < 0.05 else "No significant difference"),
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        xanchor='left', yanchor='top',
        showarrow=False,
        bgcolor='lightyellow' if p_value < 0.05 else 'lightgray',
        bordercolor='black',
        font=dict(size=11)
    )
    
    fig.update_yaxes(title_text="Plant Height (cm)")
    fig.update_xaxes(title_text="Treatment", row=1, col=2)
    
    fig.update_layout(
        title="📊 One-Way ANOVA: Plant Height vs Fertilizer Type<br><sub>Comparing 4 treatment groups</sub>",
        height=500,
        template='plotly_white'
    )
    
    return fig, df, f_stat, p_value, eta_squared, ssb, ssw, sst

# Run example
fig_anova, df_anova, f_stat, p_value, eta_sq, ssb, ssw, sst = oneway_anova_example()
fig_anova.show()

# Detailed results
print("\n📊 One-Way ANOVA Results:\n")
print("   📐 ANOVA TABLE:")
print("   ─────────────────────────────────────────────────")
print("   Source          SS      df    MS       F      p-value")
print("   ─────────────────────────────────────────────────")
df_between = 3  # k - 1
df_within = 56  # N - k
msb = ssb / df_between
msw = ssw / df_within
print(f"   Between      {ssb:7.1f}    {df_between}   {msb:7.1f}  {f_stat:6.2f}   {p_value:.4f}")
print(f"   Within       {ssw:7.1f}   {df_within}   {msw:7.1f}")
print(f"   Total        {sst:7.1f}   {df_between + df_within}")
print("   ─────────────────────────────────────────────────")

print("\n   📈 INTERPRETATION:")
print(f"      F-statistic = {f_stat:.2f}")
print(f"         → Ratio of between-group to within-group variance")
print(f"      p-value = {p_value:.4f}")
if p_value < 0.001:
    print("         → HIGHLY significant (p < 0.001)")
elif p_value < 0.05:
    print("         → Significant (p < 0.05)")
else:
    print("         → Not significant (p ≥ 0.05)")
    
print(f"\n      Effect size (η²) = {eta_sq:.3f}")
print(f"         → Fertilizer type explains {eta_sq*100:.1f}% of height variation")
if eta_sq >= 0.14:
    print("         → LARGE effect")
elif eta_sq >= 0.06:
    print("         → MEDIUM effect")
else:
    print("         → SMALL effect")

print("\n   💡 CONCLUSION:")
print("      There IS a significant difference in plant height")
print("      across the 4 fertilizer treatments.")
print("\n      ⚠️ BUT: ANOVA only tells us that SOME difference exists.")
print("      It doesn't tell us WHICH groups differ!")
print("      → Need post-hoc tests for pairwise comparisons.")

# Group means
print("\n   📊 GROUP MEANS:")
group_stats = df_anova.groupby('Treatment')['Height'].agg(['mean', 'std', 'count'])
for treatment in group_stats.index:
    mean = group_stats.loc[treatment, 'mean']
    std = group_stats.loc[treatment, 'std']
    n = group_stats.loc[treatment, 'count']
    print(f"      {treatment:15s}: {mean:5.2f} ± {std:4.2f} cm (n={int(n)})")

---

## 🔍 Part 3: Post-Hoc Tests

### The Problem:

**ANOVA tells you**: At least one group differs

**ANOVA does NOT tell you**: Which specific groups differ

### Solution: Post-Hoc Tests

**Post-hoc tests**: Pairwise comparisons AFTER significant ANOVA

**Important**: Only do post-hoc tests if ANOVA is significant!

### Common Post-Hoc Tests:

#### **1. Tukey's HSD** (Honestly Significant Difference)
```
Most popular
Controls family-wise error rate
Compares all pairwise combinations
Moderate power
```

#### **2. Bonferroni Correction**
```
Very conservative
Adjusts α for number of comparisons
α_adjusted = α / number of comparisons
Low power (too strict)
```

#### **3. Scheffé's Test**
```
Most conservative
Good for unequal sample sizes
Can test any contrast (not just pairwise)
Lowest power
```

#### **4. Dunnett's Test**
```
Compares all groups to ONE control
More powerful than Tukey's (fewer comparisons)
Use when you have a clear control group
```

### Tukey's HSD Formula:

**Critical difference**:
```
HSD = q × √(MSW/n)
```

Where:
- **q** = Studentized range statistic
- **MSW** = Mean square within groups
- **n** = Sample size per group

**If |Mean₁ - Mean₂| > HSD**: Groups differ significantly

### Reporting Post-Hoc Results:

**Compact letter display**:
```
Group        Mean    
Control      25.3  a
Organic      30.1  b
NPK          35.2  c
Slow-release 32.0  bc

Groups with same letter: Not significantly different
Groups with different letters: Significantly different
```

In [ ]:
# Post-hoc tests: Tukey's HSD
def posthoc_tukey(df):
    """
    Perform Tukey's HSD test
    """
    # Perform Tukey's HSD
    tukey_result = pairwise_tukeyhsd(
        endog=df['Height'],
        groups=df['Treatment'],
        alpha=0.05
    )
    
    # Convert to DataFrame for easier handling
    tukey_df = pd.DataFrame(data=tukey_result.summary().data[1:], 
                           columns=tukey_result.summary().data[0])
    
    # Create visualization
    fig = go.Figure()
    
    # Get unique groups and their means
    groups = df['Treatment'].unique()
    group_means = df.groupby('Treatment')['Height'].mean().sort_values(ascending=False)
    
    # Create matrix for visualization
    n_groups = len(groups)
    sig_matrix = np.zeros((n_groups, n_groups))
    
    # Fill significance matrix
    for idx, row in tukey_df.iterrows():
        group1 = row['group1']
        group2 = row['group2']
        reject = row['reject']
        
        i = list(group_means.index).index(group1)
        j = list(group_means.index).index(group2)
        
        sig_matrix[i, j] = 1 if reject == 'True' or reject == True else 0
        sig_matrix[j, i] = sig_matrix[i, j]
    
    # Create heatmap
    fig.add_trace(go.Heatmap(
        z=sig_matrix,
        x=list(group_means.index),
        y=list(group_means.index),
        colorscale=[
            [0, 'lightgray'],  # Not significant
            [1, 'red']         # Significant
        ],
        showscale=False,
        text=sig_matrix,
        texttemplate='%{text:.0f}',
        hovertemplate='%{y} vs %{x}<br>Significant: %{z}<extra></extra>'
    ))
    
    fig.update_layout(
        title="🔍 Tukey's HSD Post-Hoc Test Results<br><sub>Red = Significant difference (p < 0.05) | Gray = No difference</sub>",
        xaxis_title="Treatment",
        yaxis_title="Treatment",
        height=500,
        template='plotly_white'
    )
    
    return fig, tukey_result, tukey_df

# Run post-hoc test
fig_tukey, tukey_result, tukey_df = posthoc_tukey(df_anova)
fig_tukey.show()

# Display results
print("\n🔍 Tukey's HSD Post-Hoc Test Results:\n")
print("   📊 PAIRWISE COMPARISONS:")
print("   ─────────────────────────────────────────────────────────")
print("   Comparison               Mean Diff    p-adj    Significant?")
print("   ─────────────────────────────────────────────────────────")

for idx, row in tukey_df.iterrows():
    group1 = row['group1']
    group2 = row['group2']
    meandiff = float(row['meandiff'])
    p_adj = float(row['p-adj'])
    reject = row['reject']
    
    sig = "YES ✓" if (reject == 'True' or reject == True) else "NO"
    print(f"   {group1:12s} vs {group2:12s}  {meandiff:6.2f}     {p_adj:.4f}      {sig}")

print("   ─────────────────────────────────────────────────────────")

# Compact letter display
print("\n   📋 COMPACT LETTER DISPLAY:")
print("   ─────────────────────────────")
print("   Treatment        Mean (cm)  ")
print("   ─────────────────────────────")
group_means = df_anova.groupby('Treatment')['Height'].mean().sort_values(ascending=False)
# Simplified letter assignment (manual for clarity)
letters = {'NPK': 'a', 'Slow-release': 'ab', 'Organic': 'b', 'Control': 'c'}
for treatment in group_means.index:
    mean = group_means[treatment]
    letter = letters.get(treatment, '')
    print(f"   {treatment:15s}  {mean:5.2f}      {letter}")
print("   ─────────────────────────────")
print("   Note: Groups sharing a letter are not significantly different")

print("\n   💡 INTERPRETATION:")
print("      • NPK fertilizer produces tallest plants (significantly > Control)")
print("      • Slow-release intermediate (not different from NPK or Organic)")
print("      • Organic better than Control")
print("      • Control produces shortest plants")
print("\n   🎯 PRACTICAL RECOMMENDATION:")
print("      Use NPK for maximum growth, but Slow-release is")
print("      a good alternative if sustained release is desired.")

---

## 🎛️ Part 4: Two-Way ANOVA

### The Model:

```
Yᵢⱼₖ = μ + αᵢ + βⱼ + (αβ)ᵢⱼ + εᵢⱼₖ
```

Where:
- **αᵢ** = Effect of Factor A (level i)
- **βⱼ** = Effect of Factor B (level j)
- **(αβ)ᵢⱼ** = Interaction effect
- **εᵢⱼₖ** = Random error

### Three Research Questions:

#### **1. Main Effect of Factor A**
```
Do levels of Factor A differ (averaging over Factor B)?
```

#### **2. Main Effect of Factor B**
```
Do levels of Factor B differ (averaging over Factor A)?
```

#### **3. Interaction A × B**
```
Does the effect of A depend on the level of B?
(Or vice versa)
```

### Understanding Interactions:

**No Interaction** (parallel lines):
```
      │
   30 │     ╱╱
      │   ╱╱
   20 │ ╱╱
      │╱
   10 │─────────
        Low  High
      Factor B

Effect of A same at all levels of B
```

**Interaction Present** (non-parallel lines):
```
      │
   30 │     ╱
      │    ╱
   20 │   ╱
      │  ╲
   10 │   ╲
        Low  High
      Factor B

Effect of A CHANGES at different levels of B
```

### ANOVA Table for Two-Way:

| Source | SS | df | MS | F | p |
|--------|----|----|-------|---|---|
| **Factor A** | SSₐ | a-1 | MSₐ | Fₐ | pₐ |
| **Factor B** | SSᵦ | b-1 | MSᵦ | Fᵦ | pᵦ |
| **A × B** | SSₐᵦ | (a-1)(b-1) | MSₐᵦ | Fₐᵦ | pₐᵦ |
| **Error** | SSₑ | ab(n-1) | MSₑ | | |
| **Total** | SSₜ | abn-1 | | | |

### Interpreting Results:

**If interaction significant**:
- Interpret interaction first!
- Main effects may be misleading
- Describe simple effects (effect of A at each level of B)

**If interaction not significant**:
- Interpret main effects
- Effects are additive

In [ ]:
# Two-way ANOVA example: Fertilizer × Watering
def twoway_anova_example(n_per_cell=10, seed=42):
    """
    Plant growth with 2 factors:
    Factor A: Fertilizer (Control, NPK)
    Factor B: Water (Low, High)
    With interaction!
    """
    np.random.seed(seed)
    
    # Design: 2×2 factorial
    # Interaction: High water helps MORE with NPK than with Control
    conditions = [
        ('Control', 'Low', 20),
        ('Control', 'High', 25),  # Water adds +5
        ('NPK', 'Low', 28),
        ('NPK', 'High', 40),      # Water adds +12 (interaction!)
    ]
    
    data = []
    for fertilizer, water, mean in conditions:
        heights = np.random.normal(mean, 3, n_per_cell)
        for height in heights:
            data.append({
                'Fertilizer': fertilizer,
                'Water': water,
                'Height': height
            })
    
    df = pd.DataFrame(data)
    
    # Perform two-way ANOVA using statsmodels
    model = ols('Height ~ C(Fertilizer) * C(Water)', data=df).fit()
    anova_table = anova_lm(model, typ=2)
    
    # Create interaction plot
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            'Interaction Plot',
            'Cell Means'
        ),
        horizontal_spacing=0.15
    )
    
    # Interaction plot
    for fert in ['Control', 'NPK']:
        subset = df[df['Fertilizer'] == fert]
        means = subset.groupby('Water')['Height'].mean()
        
        fig.add_trace(
            go.Scatter(
                x=['Low', 'High'],
                y=[means['Low'], means['High']],
                mode='lines+markers',
                line=dict(width=3),
                marker=dict(size=12),
                name=fert
            ),
            row=1, col=1
        )
    
    # Bar plot
    cell_means = df.groupby(['Fertilizer', 'Water'])['Height'].mean()
    cell_labels = [f"{f}\n{w}" for f, w in cell_means.index]
    colors = ['lightblue', 'lightcoral', 'lightgreen', 'gold']
    
    fig.add_trace(
        go.Bar(
            x=cell_labels,
            y=cell_means.values,
            marker_color=colors,
            showlegend=False
        ),
        row=1, col=2
    )
    
    fig.update_xaxes(title_text="Water Level", row=1, col=1)
    fig.update_xaxes(title_text="Fertilizer × Water", row=1, col=2)
    fig.update_yaxes(title_text="Plant Height (cm)")
    
    fig.update_layout(
        title="🎛️ Two-Way ANOVA: Fertilizer × Water Interaction<br><sub>Non-parallel lines indicate interaction</sub>",
        height=500,
        template='plotly_white'
    )
    
    return fig, df, anova_table

# Run example
fig_2way, df_2way, anova_2way = twoway_anova_example()
fig_2way.show()

# Display ANOVA table
print("\n🎛️ Two-Way ANOVA Results:\n")
print("   📊 ANOVA TABLE:")
print(anova_2way.to_string())

print("\n   🔍 INTERPRETATION:")
print("\n   1️⃣ MAIN EFFECT: Fertilizer")
f_fert = anova_2way.loc['C(Fertilizer)', 'F']
p_fert = anova_2way.loc['C(Fertilizer)', 'PR(>F)']
print(f"      F = {f_fert:.2f}, p = {p_fert:.4f}")
if p_fert < 0.05:
    print("      ✓ Significant: NPK increases height vs Control")
else:
    print("      ✗ Not significant")

print("\n   2️⃣ MAIN EFFECT: Water")
f_water = anova_2way.loc['C(Water)', 'F']
p_water = anova_2way.loc['C(Water)', 'PR(>F)']
print(f"      F = {f_water:.2f}, p = {p_water:.4f}")
if p_water < 0.05:
    print("      ✓ Significant: High water increases height vs Low")
else:
    print("      ✗ Not significant")

print("\n   3️⃣ INTERACTION: Fertilizer × Water")
f_int = anova_2way.loc['C(Fertilizer):C(Water)', 'F']
p_int = anova_2way.loc['C(Fertilizer):C(Water)', 'PR(>F)']
print(f"      F = {f_int:.2f}, p = {p_int:.4f}")
if p_int < 0.05:
    print("      ✓ SIGNIFICANT INTERACTION!")
    print("      → Effect of water DEPENDS on fertilizer type")
    print("\n      📈 Simple Effects:")
    print("         Control:  High water adds ~5 cm")
    print("         NPK:      High water adds ~12 cm")
    print("\n      💡 Practical Meaning:")
    print("         NPK fertilizer is MUCH more effective")
    print("         when combined with high watering!")
    print("         Synergistic effect → maximize both factors")
else:
    print("      ✗ No interaction: Effects are additive")

print("\n   🎯 RECOMMENDATION:")
print("      For maximum growth: Use NPK + High watering")
print("      This combination produces synergistic benefits!")

---

## ✅ Part 5: ANOVA Assumptions

### The Four Assumptions:

#### **1. Independence**
```
Observations are independent
Check: Consider study design
Violated by: Pseudoreplication, clustering
```

#### **2. Normality**
```
Residuals are normally distributed WITHIN each group
Check: Q-Q plots, Shapiro-Wilk test
Note: ANOVA robust to violations with large n
```

#### **3. Homogeneity of Variance** (Homoscedasticity)
```
Equal variance across groups
Check: Levene's test, Bartlett's test
Most critical assumption!
```

#### **4. No Outliers**
```
Extreme values can distort results
Check: Box plots, residual plots
```

### Testing Assumptions:

**Levene's Test** (homogeneity of variance):
```
H₀: Variances are equal across groups
If p < 0.05: Variances differ (assumption violated)
```

**Shapiro-Wilk Test** (normality):
```
H₀: Data are normally distributed
If p < 0.05: Not normal (assumption violated)
```

### What to Do if Assumptions Violated:

**Non-normality**:
- Transform data (log, sqrt, etc.)
- Use non-parametric alternative (Kruskal-Wallis)
- With large n (>30 per group), proceed anyway (CLT)

**Unequal variances**:
- Welch's ANOVA (doesn't assume equal variances)
- Transform data
- Non-parametric test

**Outliers**:
- Investigate (data entry error? real extreme?)
- Remove if justified
- Use robust methods

### Non-Parametric Alternative:

**Kruskal-Wallis Test**:
- Rank-based (doesn't assume normality)
- Compares medians instead of means
- Less powerful than ANOVA
- Use when assumptions badly violated

In [ ]:
# Check ANOVA assumptions
def check_anova_assumptions(df, group_col='Treatment', value_col='Height'):
    """
    Test ANOVA assumptions
    """
    # 1. Homogeneity of variance (Levene's test)
    groups = [df[df[group_col] == g][value_col].values 
              for g in df[group_col].unique()]
    levene_stat, levene_p = stats.levene(*groups)
    
    # 2. Normality of residuals
    # Fit ANOVA model
    grand_mean = df[value_col].mean()
    group_means = df.groupby(group_col)[value_col].transform('mean')
    residuals = df[value_col] - group_means
    
    shapiro_stat, shapiro_p = stats.shapiro(residuals)
    
    # Create diagnostic plots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Box Plots by Group (Check Outliers)',
            'Residuals Distribution (Check Normality)',
            'Q-Q Plot (Check Normality)',
            'Residuals vs Fitted (Check Homoscedasticity)'
        ),
        vertical_spacing=0.12,
        horizontal_spacing=0.12
    )
    
    # 1. Box plots
    for group in df[group_col].unique():
        subset = df[df[group_col] == group][value_col]
        fig.add_trace(
            go.Box(y=subset, name=group, showlegend=False),
            row=1, col=1
        )
    
    # 2. Histogram of residuals
    fig.add_trace(
        go.Histogram(x=residuals, nbinsx=20, 
                    marker_color='lightblue',
                    showlegend=False),
        row=1, col=2
    )
    
    # 3. Q-Q plot
    sorted_residuals = np.sort(residuals)
    n = len(sorted_residuals)
    theoretical_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, n))
    
    # Standardize
    std_residuals = (sorted_residuals - np.mean(sorted_residuals)) / np.std(sorted_residuals)
    
    fig.add_trace(
        go.Scatter(x=theoretical_quantiles, y=std_residuals,
                  mode='markers', marker=dict(color='blue'),
                  showlegend=False),
        row=2, col=1
    )
    # Reference line
    fig.add_trace(
        go.Scatter(x=[-3, 3], y=[-3, 3],
                  mode='lines', line=dict(color='red', dash='dash'),
                  showlegend=False),
        row=2, col=1
    )
    
    # 4. Residuals vs Fitted
    fitted_values = group_means
    fig.add_trace(
        go.Scatter(x=fitted_values, y=residuals,
                  mode='markers', marker=dict(color='green'),
                  showlegend=False),
        row=2, col=2
    )
    fig.add_hline(y=0, line_dash="dash", line_color="red", row=2, col=2)
    
    # Update axes
    fig.update_yaxes(title_text=value_col, row=1, col=1)
    fig.update_xaxes(title_text="Residuals", row=1, col=2)
    fig.update_yaxes(title_text="Frequency", row=1, col=2)
    fig.update_xaxes(title_text="Theoretical Quantiles", row=2, col=1)
    fig.update_yaxes(title_text="Standardized Residuals", row=2, col=1)
    fig.update_xaxes(title_text="Fitted Values", row=2, col=2)
    fig.update_yaxes(title_text="Residuals", row=2, col=2)
    
    fig.update_layout(
        title="✅ ANOVA Assumption Checks<br><sub>Diagnostic plots for assumptions</sub>",
        height=800,
        template='plotly_white'
    )
    
    return fig, levene_stat, levene_p, shapiro_stat, shapiro_p

# Check assumptions on our data
fig_assumptions, lev_stat, lev_p, shap_stat, shap_p = check_anova_assumptions(df_anova)
fig_assumptions.show()

print("\n✅ ANOVA Assumption Tests:\n")
print("   1️⃣ HOMOGENEITY OF VARIANCE (Levene's Test):")
print(f"      Statistic = {lev_stat:.4f}")
print(f"      p-value = {lev_p:.4f}")
if lev_p > 0.05:
    print("      ✓ Assumption MET: Variances are equal (p > 0.05)")
else:
    print("      ✗ Assumption VIOLATED: Variances differ (p < 0.05)")
    print("      → Consider Welch's ANOVA or transformation")

print("\n   2️⃣ NORMALITY OF RESIDUALS (Shapiro-Wilk Test):")
print(f"      Statistic = {shap_stat:.4f}")
print(f"      p-value = {shap_p:.4f}")
if shap_p > 0.05:
    print("      ✓ Assumption MET: Residuals normally distributed (p > 0.05)")
else:
    print("      ✗ Assumption VIOLATED: Residuals not normal (p < 0.05)")
    print("      → Consider transformation or Kruskal-Wallis test")

print("\n   3️⃣ INDEPENDENCE:")
print("      ✓ Assumed based on experimental design")
print("      (Each plant is independent unit)")

print("\n   4️⃣ OUTLIERS:")
print("      Check box plots (top-left)")
print("      No extreme outliers apparent")

print("\n   💡 OVERALL ASSESSMENT:")
if lev_p > 0.05 and shap_p > 0.05:
    print("      ✅ All assumptions satisfied")
    print("      → ANOVA results are reliable!")
else:
    print("      ⚠️ Some assumptions may be violated")
    print("      → Results should be interpreted with caution")
    print("      → Consider alternative analyses")

---

## 🎓 Summary

### Key Concepts:

✅ **ANOVA**: Compare means across 3+ groups  
✅ **One-Way**: One categorical factor  
✅ **F-ratio**: Between-group variance / Within-group variance  
✅ **Post-hoc**: Pairwise comparisons after significant ANOVA  
✅ **Two-Way**: Two factors, can test interactions  
✅ **Interaction**: Effect of one factor depends on another  
✅ **Assumptions**: Independence, normality, homogeneity, no outliers  
✅ **Effect size**: η² shows proportion of variance explained  

### Decision Tree:

```
How many groups?
  ├─ 2 groups → Use t-test
  └─ 3+ groups → ANOVA
       │
       ├─ One factor → One-way ANOVA
       │    ├─ Significant? → Post-hoc tests (Tukey's)
       │    └─ Not significant? → No group differences
       │
       ├─ Two factors → Two-way ANOVA
       │    ├─ Interaction significant? → Interpret interaction
       │    └─ No interaction? → Interpret main effects
       │
       └─ Assumptions violated? → Non-parametric (Kruskal-Wallis)
```

### ANOVA vs Regression:

| Feature | ANOVA | Regression |
|---------|-------|------------|
| **X type** | Categorical | Continuous |
| **Purpose** | Compare group means | Model relationship |
| **Output** | Group differences | Equation |
| **Test statistic** | F | F or t |

**Note**: ANOVA and regression are mathematically equivalent (both are General Linear Models)!

### Common Mistakes:

❌ **Multiple t-tests** instead of ANOVA  
❌ **Post-hoc without significant ANOVA**  
❌ **Ignoring assumptions**  
❌ **Misinterpreting interactions**  
❌ **Confusing significance with importance**  
❌ **Not reporting effect size**  

### Reporting ANOVA Results:

**Minimal**:
```
"Fertilizer type significantly affected plant height 
(F₃,₅₆ = 25.3, p < 0.001, η² = 0.58)."
```

**Complete**:
```
Methods: "We used one-way ANOVA to compare plant height 
across four fertilizer treatments (n = 15 per group)."

Results: "Fertilizer type significantly affected plant height 
(F₃,₅₆ = 25.3, p < 0.001, η² = 0.58). Post-hoc Tukey's HSD 
tests revealed that NPK fertilizer (35.2 ± 1.0 cm) produced 
significantly taller plants than all other treatments 
(all p < 0.01)."
```

### Best Practices:

**1. Plan Ahead**: Design study with ANOVA in mind  
**2. Check Assumptions**: Run diagnostic tests  
**3. Equal Sample Sizes**: Makes analysis more robust  
**4. Report Effect Size**: Not just p-value  
**5. Visualize**: Always plot the data  
**6. Post-Hoc Carefully**: Only if ANOVA significant  
**7. Interpret Interactions**: Don't ignore them!  

### Formula Summary:

**F-statistic**:
```
F = MSB / MSW
```

**Effect size**:
```
η² = SSB / SST
```

**Degrees of freedom**:
```
df_between = k - 1
df_within = N - k
```

---

<div align="center">

**Made with 💚 by Ms. Susama Kar & Dr. Alok Patel**

[📓 Previous: Regression Analysis](08_regression_analysis.ipynb) | 
[🏠 Unit 4 Home](../../)

**🎉 Congratulations! You've completed Unit 4: Biometry! 🎉**

</div>